In [1]:
import os
from pathlib_mate import Path

import boto3
import polars as pl
from s3pathlib import S3Path, context
from boto_session_manager import BotoSesManager
from awscli_mate.api import AWSCliConfig

dir_here = Path.cwd().absolute()

aws_profile_dev = "bmt_app_dev_us_east_1"
aws_profile_test = "bmt_app_test_us_east_1"

awscli_config = AWSCliConfig()
awscli_config.set_profile_as_default(aws_profile_test)

bsm = BotoSesManager(profile_name=aws_profile_dev)
bsm_test = BotoSesManager(profile_name=aws_profile_test)

context.attach_boto_session(boto_ses=bsm.boto_ses)
bucket = f"{bsm.aws_account_alias}-{bsm.aws_region}-data"
s3dir_root = S3Path(
    f"s3://{bucket}/projects/learn_polars"
    f"/input-and-output/local-and-cloud/"
).to_dir()

In [2]:
n_rows = 1000
df = pl.DataFrame(
    {
        "id": list(range(1, 1 + n_rows)),
        "name": ["Alice"] * n_rows,
        # "details": [{"dob": "2000-01-01"}] * n_rows
    }
)
df

id,name
i64,str
1,"""Alice"""
2,"""Alice"""
3,"""Alice"""
4,"""Alice"""
5,"""Alice"""
…,…
996,"""Alice"""
997,"""Alice"""
998,"""Alice"""


## Local File IO

In [3]:
def clean_up_local():
    for f in dir_here.glob("*.json"):
        f.unlink()

### Write to Local File

**额外依赖**

polars 无需任何额外依赖就可以对本地文件进行读写.

**节约内存**

在很多 IO 场景中都有用数据流, 而不是一次性读取全部数据的方式节约内存. 这种方式通常是用牺牲性能换取更小的内存开销. 典型的使用场景如下:

- 逐行读取大文件, 逐行对数据进行处理, 并将处理后的数据逐行写入新文件.

当你大的 DataFrame 写入本地文件时, Python 默认的 IO 就是流式写入, 所以你无需担心内存问题.

**参数说明**

``polars.write_parquet`` 方法接受:

1. 字符串: 字符串将会被视为本地文件的路径. 
2. File-liked Object: 你可以用 File-liked Object 作为输入也可以.

In [4]:
clean_up_local()

# Write to Local File
path = dir_here / "data.json"
df.write_ndjson(str(path))
print(f"{path.exists() = }, {path.size_in_text = }")

path.exists() = True, path.size_in_text = '25.29 KB'


### Read from Local File

In [5]:
# Read from Local File
pl.read_ndjson(str(path))

id,name
i64,str
1,"""Alice"""
2,"""Alice"""
3,"""Alice"""
4,"""Alice"""
5,"""Alice"""
…,…
996,"""Alice"""
997,"""Alice"""
998,"""Alice"""


In [6]:
with path.open("rb") as f:
    df = pl.read_ndjson(f)
df

id,name
i64,str
1,"""Alice"""
2,"""Alice"""
3,"""Alice"""
4,"""Alice"""
5,"""Alice"""
…,…
996,"""Alice"""
997,"""Alice"""
998,"""Alice"""


### Read from Multiple Files

This example shows how to work with multiple files.

``polars`` doesn't support automatically write big dataframe into multiple small files. You have to divide the dataframe into small ones and write them one by one. 

If you really want to do that, you can check [pyarrow.dataset.write_dataset](https://arrow.apache.org/docs/python/generated/pyarrow.dataset.write_dataset.html)

Polars lazy API has a set of functions [polars.scan_${data_format}](https://docs.pola.rs/api/python/stable/search.html?q=polars.scan_) to read data from multiple files.

In [7]:
clean_up_local()

# Write to Local File
for i in range(1, 1+3):
    path = dir_here / f"data{i}.json"
    df.write_ndjson(str(path))

# Read from Local File
pl.scan_ndjson(f"{dir_here}/*.json").collect()

id,name
i64,str
1,"""Alice"""
2,"""Alice"""
3,"""Alice"""
4,"""Alice"""
5,"""Alice"""
…,…
996,"""Alice"""
997,"""Alice"""
998,"""Alice"""


## Cloud Storage IO

- 对于 Write 操作, 如果你要用 [Cloud storage](https://docs.pola.rs/user-guide/io/cloud-storage/) 官方文档中的 API, 那么 polars 依赖于 ``fsspec s3fs (for AWS) adlfs (for Azure) gcsfs (for GCP)`` 这些额外的依赖来提供 IO 层的抽象以及异步 API 调用. **但如果你愿意手动用** [io](https://docs.python.org/3/library/io.html) **库先将 binary 数据写入 buffer, 然后再用各个 Cloud 原生 API 将数据写入, 又或是先写入到本地文件后再用各个 Cloud 原生 API 上传, 那么你可以不依赖于 fsspec 系列库**.
- 对于 Read 操作, polars 不依赖 ``fsspec`` 系列库 (也就间接不依赖于 ``aiobotocore`` 等库). 例如 [read_parquet](https://docs.pola.rs/api/python/stable/reference/api/polars.read_parquet.html), [scan_parquet](https://docs.pola.rs/api/python/stable/reference/api/polars.scan_parquet.html) (其他文件格式同理) 都有 ``storage_options`` 这个参数可供你控制 credentials, 以及底层的 API 调用.

我其实不理解为什么 Write 操作不能像 Read 一样, 不依赖任何除了 Cloud 原生 SDK 以外的任何库, 可能会在新版本加入这一支持吧.

Reference:

- [Cloud storage](https://docs.pola.rs/user-guide/io/cloud-storage/)

In [8]:
def clean_up_s3():
    s3dir_root.delete()

### Write to AWS S3

#### Method 1, use [io.BytesIO](https://docs.python.org/3/library/io.html#binary-i-o) 

In [9]:
# clean_up_s3()

import io

s3path_ndjson = s3dir_root / "data.json"
buffer = io.BytesIO()
df.write_ndjson(buffer)
print(f"Write to {s3path_ndjson.uri}")
s3path_ndjson.write_bytes(buffer.getvalue())
print(f"preview at: {s3path_ndjson.console_url}")

Write to s3://bmt-app-dev-us-east-1-data/projects/learn_polars/input-and-output/local-and-cloud/data.json
preview at: https://console.aws.amazon.com/s3/object/bmt-app-dev-us-east-1-data?prefix=projects/learn_polars/input-and-output/local-and-cloud/data.json


#### Method 2, write to local file then upload

In [10]:
# clean_up_s3()

path_parquet = dir_here / "data.parquet"
s3path_parquet = s3dir_root / "data.parquet"
print(f"Write to {path.basename!r}")
df.write_parquet(str(path_parquet))
print(f"Upload to {s3path_parquet.uri}")
bsm.s3_client.upload_file(str(path_parquet), Bucket=bucket, Key=s3path_parquet.key)
print(f"preview at: {s3path_ndjson.console_url}")

Write to 'data3.json'
Upload to s3://bmt-app-dev-us-east-1-data/projects/learn_polars/input-and-output/local-and-cloud/data.parquet
preview at: https://console.aws.amazon.com/s3/object/bmt-app-dev-us-east-1-data?prefix=projects/learn_polars/input-and-output/local-and-cloud/data.json


#### Method 3, use s3fs

In [12]:
import s3fs

# clean_up_s3()
credentials = bsm.boto_ses.get_credentials()

fs = s3fs.S3FileSystem(
    key=credentials.access_key,
    secret=credentials.secret_key,
)
s3path_csv = s3dir_root / "data.csv"
print(f"Write to {s3path_csv.uri}")
with fs.open(s3path_csv.uri, mode="wb") as f:
    df.write_csv(f)
print(f"preview at: {s3path_csv.console_url}")

Write to s3://bmt-app-dev-us-east-1-data/projects/learn_polars/input-and-output/local-and-cloud/data.csv
preview at: https://console.aws.amazon.com/s3/object/bmt-app-dev-us-east-1-data?prefix=projects/learn_polars/input-and-output/local-and-cloud/data.csv


/var/folders/bl/vkmgjdsx5115w2xcnp67_8y40000gn/T/ipykernel_15112/4035855381.py:13: UserWarning: Polars found a filename. Ensure you pass a path to the file instead of a python file object when possible for best performance.
  df.write_csv(f)


### Read from AWS S3

In [13]:
def get_storage_options(boto_ses: "boto3.Session") -> dict:
    credential = boto_ses.get_credentials()
    storage_options = {
        "aws_region": boto_ses.region_name,
        "aws_access_key_id": credential.access_key,
        "aws_secret_access_key": credential.secret_key,
    }
    if credential.token:
        storage_options["aws_session_token"] = credential.token
    return storage_options


#### Read NDJSON

In [25]:
# This won't work because the default credential is bmt_app_test, 
# but the s3 object is on bmt_app_dev
try:
    df = pl.read_ndjson(s3path_ndjson.uri)
except Exception as e:
    print(e)

Generic S3 error: Client error with status 403 Forbidden: No Body


In [41]:
# This doesn't work
storage_options = get_storage_options(bsm.boto_ses)
df = pl.read_ndjson(
    s3path_ndjson.uri,
    storage_options=storage_options,
)
df

ComputeError: Generic S3 error: Client error with status 403 Forbidden: No Body

In [28]:
# This doesn't work
storage_options = get_storage_options(bsm.boto_ses)
df = pl.scan_ndjson(
    s3path_ndjson.uri,
    storage_options=storage_options,
).collect()
df

ComputeError: Generic S3 error: Client error with status 403 Forbidden: No Body

In [36]:
# This won't work
credential = bsm.boto_ses.get_credentials()
os.environ["AWS_REGION"] = bsm.aws_region
os.environ["AWS_ACCESS_KEY_ID"] = credential.access_key
os.environ["AWS_SECRET_ACCESS_KEY"] = credential.secret_key
try:
    df = pl.read_parquet(
        s3path_ndjson.uri,
    )
except Exception as e:
    print(e)

Generic S3 error: Client error with status 403 Forbidden: No Body


#### Read Parquet

**结论**

- scan_parquet 和 read_parquet 都能用, 但推荐使用 scan_parquet.

In [37]:
# This won't work because the default credential is bmt_app_test, 
# but the s3 object is on bmt_app_dev
try:
    df = pl.read_parquet(s3path_parquet.uri)
except Exception as e:
    print(e)

Generic S3 error: Client error with status 403 Forbidden: No Body


In [38]:
# This would work
storage_options = get_storage_options(bsm.boto_ses)
df = pl.read_parquet(
    s3path_parquet.uri,
    storage_options=storage_options,
)
df

id,name,details
i64,str,struct[1]
1,"""Alice""","{""2000-01-01""}"
2,"""Alice""","{""2000-01-01""}"
3,"""Alice""","{""2000-01-01""}"
4,"""Alice""","{""2000-01-01""}"
5,"""Alice""","{""2000-01-01""}"
…,…,…
996,"""Alice""","{""2000-01-01""}"
997,"""Alice""","{""2000-01-01""}"
998,"""Alice""","{""2000-01-01""}"


In [24]:
# This would work
storage_options = get_storage_options(bsm.boto_ses)
df = pl.scan_parquet(
    s3path_parquet.uri,
    storage_options=storage_options,
).collect()
df

id,name
i64,str
1,"""Alice"""
2,"""Alice"""
3,"""Alice"""
4,"""Alice"""
5,"""Alice"""
…,…
996,"""Alice"""
997,"""Alice"""
998,"""Alice"""


In [39]:
# This won't work
credential = bsm.boto_ses.get_credentials()
os.environ["AWS_REGION"] = bsm.aws_region
os.environ["AWS_ACCESS_KEY_ID"] = credential.access_key
os.environ["AWS_SECRET_ACCESS_KEY"] = credential.secret_key
try:
    df = pl.read_parquet(
        s3path_parquet.uri,
    )
except Exception as e:
    print(e)

Generic S3 error: Client error with status 403 Forbidden: No Body


#### Read CSV

结论:

- 推荐使用 scan_csv 而不是 read_csv. read_csv 对 storage options 的支持不好.

In [20]:
# This won't work because the default credential is bmt_app_test, 
# but the s3 object is on bmt_app_dev
try:
    df = pl.read_csv(s3path_csv.uri)
    print(df)
except Exception as e:
    print(e)

Forbidden


In [22]:
# This doesn't work
try:
    storage_options = get_storage_options(bsm.boto_ses)
    df = pl.read_csv(
        s3path_csv.uri,
        storage_options=storage_options,
    )
    df
except Exception as e:
    print(e)

AioSession.__init__() got an unexpected keyword argument 'aws_region'


In [23]:
# This would work
storage_options = get_storage_options(bsm.boto_ses)
df = pl.scan_csv(
    s3path_csv.uri,
    storage_options=storage_options,
).collect()
df

id,name
i64,str
1,"""Alice"""
2,"""Alice"""
3,"""Alice"""
4,"""Alice"""
5,"""Alice"""
…,…
996,"""Alice"""
997,"""Alice"""
998,"""Alice"""
